In [29]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import requests
import json
API_TOKEN  = r'GRjdYvo48ghthisisafaketokenlol'

In [132]:
# PUT requests to change the sales status of existiong jobs
def update_sales_status(API_TOKEN, uuid, name, status):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        "sales_status": status
        # 'contract_date': "12/08/2023"
    }
    response = requests.put(url, params=params, data = payload)
     
    if response.status_code == 200:
        # print("Data:")
       
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        # print(len(df))
    else:
        print(f"Error {response.status_code}: {response.text}")


def update_contract_date(API_TOKEN, uuid, name, new_date):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        'contract_date': new_date
    }
    response = requests.put(url, params=params, data = payload)
     
    if response.status_code == 200:
        # print("Data:")
       
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        # print(len(df))
    else:
        print(f"Error {response.status_code}: {response.text}")


In [ ]:
# data set of old budgets to become 
df2 = df
df2['bid_date'] = pd.to_datetime(df2['bid_date'], format='%m/%d/%Y')
in_range = df2[df2['bid_date'] < pd.to_datetime("2024-12-31")]
# I only want budgets
just_buds = in_range[in_range['sales_status'] == 'Budget']
len(just_buds)
# in_range = df[df['']]
for ind in range(0,len(just_buds)):
    current = just_buds.iloc[ind]
    # print(current['uuid'], current['name'])
    update_sales_status(API_TOKEN, current['uuid'], current['name'], 'Budget Loss')
    print(ind)

In [ ]:
df3 = df
df3['bid_date'] = pd.to_datetime(df3['bid_date'], format='%m/%d/%Y')
df3['dead_date'] = pd.to_datetime(df3['dead_date']).dt.date
# df3['dead_date'] = df3['dead_date'].astype('datetime64[D]')
in_range = df3[df3['bid_date'] < pd.to_datetime("2024-10-15")]
# I only want budgets
just_wins = in_range[in_range['sales_status'] == 'Won']
need_contract_date = just_wins.dropna(subset=['dead_date'])
# need_contract_date = just_wins[just_wins['dead_date'] != pd.NaT]
print(len(just_wins), len(need_contract_date))
for ind in range(0,len(need_contract_date)):
    current = need_contract_date.iloc[ind]
    # print(current['uuid'], current['name'], str(current['dead_date']))
    update_contract_date(API_TOKEN, current['uuid'], current['name'], current['dead_date'])
    print(ind)


In [ ]:
# creat a test subset 
test_subset = need_contract_date[0:5] 
test_subset

In [97]:
%%time 
# data set of projects from followup crm

url = 'https://app.followupcrm.com/api/v1/file-data'
params = {
    'api_token': API_TOKEN,
    'per_page': 465
}
 
response = requests.get(url, params=params)
 
if response.status_code == 200:
    # print("Data:")
   
    json_data = response.json()
    df = pd.json_normalize(json_data['data'])
    print(len(df))
 
else:
    print(f"Error {response.status_code}: {response.text}")

464
CPU times: total: 62.5 ms
Wall time: 59.4 s


In [7]:
df.columns

Index(['id', 'uuid', 'team_id', 'name', 'address', 'city', 'state', 'zip',
       'sales_behaviours', 'account_manager', 'estimator', 'companies',
       'proposal_delivered', 'sales_status', 'office', 'Lead_From',
       'contract_amount', 'base_bid', 'contract_date', 'contract_type',
       'bid_date', 'last_update', 'created_at', 'external_id',
       'external_vendor', 'custom_fields', 'notes', 'outlook_conversation_id',
       'estimated_contract_date', 'gross_margin_percentage', 'labor_hours',
       'dead_date', 'marketing_status', 'pre_bid', 'take_off_person',
       'products_and_services', 'client_profile', 'relate_and_connect',
       'rules_of_engagement', 'find_problems', 'money_step',
       'decision_process', 'change_orders', 'record_detail_custom_fields',
       'contact.name', 'contact.phone', 'contact.fax', 'contact.cell',
       'contact.email'],
      dtype='object')